# Init

In [2]:
%cd ..

E:\Coding\PlasmaChemistry


In [1]:
import yaml
import re
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.integrate import (solve_ivp, ode)
from plasmistry import constants as const
from plasmistry.molecule import get_vib_energy

ModuleNotFoundError: No module named 'plasmistry'

In [4]:
from plasmistry.reactions import CoefReactions

In [5]:
np.set_printoptions(precision=1, linewidth=130, edgeitems=5)
pd.options.display.float_format = '{:,.1e}'.format

In [6]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,9)
plt.rcParams["font.size"] = 14

In [ ]:
from yaml_demo import (eval_constructor, LT_constructor, Arr_constructor,
                       H2_vib_energy_in_eV, CO2_vib_energy_in_eV,
                       CO_vib_energy_in_eV, Reaction_block,
                       Cros_Reaction_block, Coef_Reaction_block)

yaml.add_constructor(u"!eval", eval_constructor)
yaml.add_constructor(u"!LT", LT_constructor)
yaml.add_constructor(u"!Arr", Arr_constructor)

# Read reactions

In [10]:
with open(r"_yaml/test_0.yaml") as f:
    all_rctns = yaml.load(f)
rctn_considered = all_rctns[-1]['The reactions considered'][
    'electron reactions']
for _key in rctn_considered:
    print(_key)

ComposerError: found undefined alias 'CO2_dis_cs_path'
  in "_yaml/test_0.yaml", line 250, column 22

In [ ]:
coef_block = Coef_Reaction_block(rctn_dict=rctn_considered["CO2_VT_with_CO2"])

In [ ]:
coef_df = coef_block.generate_crostn_dataframe()

In [ ]:
coef_df.columns

# Instance reaction
- \__init__
- .set_pre_exec_list() (optional)
- .compile_k_str()








In [ ]:
coef = CoefReactions(reactant=coef_df['reactant'],
                    product=coef_df['product'],
                    k_str=coef_df['kstr'])

In [ ]:
print(coef)

In [ ]:
coef.compile_k_str()

 ## set rate constant
- .set_rate_const(*, Tgas_K, Te_eV, EN_Td)

In [ ]:
%timeit coef.set_rate_const(Tgas_K=1000, Te_eV=1.0, EN_Td=3)

In [ ]:
coef.rate_const

 ## set rate
- .set_rate(*, density)

In [ ]:
coef.set_rate(density=coef.get_initial_density(density_dict={'CO2':1e25,
                                                            'CO2(v1)': 1e25,
                                                            'CO2(v2)': 1e25,
                                                            'CO': 1e25}))

In [ ]:
coef.rate

## get
- .get_dn()
- .get_dH_e()
- .get_dH_g()

In [ ]:
coef.get_dn()

In [ ]:
coef.species

In [ ]:
print(coef)

# Evolve

In [ ]:
def dndt(t, y):
    coef.set_rate_const(Tgas_K=1000)
    coef.set_rate(density=y)
    return coef.get_dn()

In [ ]:
y0 = np.ones_like(coef.species)*1e20

In [ ]:
time_span = [0, 1e3]
sol = solve_ivp(dndt, time_span, y0, method="BDF", rtol=5e-2)

In [ ]:
sol.t.shape

In [ ]:
sol.y.shape

In [ ]:
plt.semilogx(sol.t, sol.y.transpose(), marker='.')

In [ ]:
sol.y[:,-1]

In [ ]:
coef.set_rate(density=sol.y[:,-1].astype(np.float))

In [ ]:
coef.view_dndt()

In [ ]:
coef.view_density(density=sol.y[:,-1])